In [ ]:
# WARNING: before running this script make sure you created the proper TCGA folder structure.
# See the notebook: createFolderStructure
# TCGA folder structure should be as follows:
#   collection -> cohorts -> cases (patient folders) -> slide folders

# This script will move slides from the main cohort folders into their corresponding slide folders, 
# leaving the cohort folders without any items (images)                

In [ ]:
#URL to the API
BASE_URL = 'http://candygram.neurology.emory.edu:8080/api/v1'

#TCGA collection name
COLLECTION_NAME = 'TCGA'

In [1]:
import json
import girder_client
import collections
import sys
sys.path.append("../utils")
from girder_utils import *

In [ ]:
gc = girder_client.GirderClient(apiUrl=BASE_URL)
gc.authenticate(interactive=True)

In [ ]:
##Going to get my collection by ID so I can interact with it
requestUrl = 'resource/lookup?path=collection/%s' % COLLECTION_NAME
collectionData = gc.getResource(requestUrl)
collectionId = collectionData['_id']

In [ ]:
#Second, for each cohort (main folder) loop through the items
#Then move the item to its corresponding patient folder
#Use folderName2Id to map patient ID to folder ID
folders = recurseGetResource(gc, collectionId, 'folder', 'collection')
print "Moving items to slide folders..."

for folder in folders:
    print "Working on " + folder["name"]
    offset = -1
    
    while True:
        items = gc.listItem(folder["_id"], limit=500, offset=offset + 1)
        for item in items:
            folderName = item["name"].split(".",1)[0]
            requestUrl = 'resource/search?q=%s&mode=prefix&types=%%5B%%22folder%%22%%5D&level=0&limit=10&offset=0' % folderName
            results = gc.getResource(requestUrl)["folder"]
           
            if len(results):
                folderId = results[0]['_id']
                try:
                    gc.put("item/" + item["_id"], {"folderId": folderId})
                except:
                    print "failed " + folderName
                    pass
            
        if len(items) < 500: break
        offset += 500
        
print "Done"